In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import plot_confusion_matrix  , accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
water = pd.read_csv("../input/water-potability/water_potability.csv") 

In [ ]:
water.head()


The water_potability.csv file contains water quality metrics for 3276 different water bodies.

### 1. pH value:
PH is an important parameter in evaluating the acid–base balance of water. It is also the indicator of acidic or alkaline condition of water status. WHO has recommended maximum permissible limit of pH from 6.5 to 8.5. The current investigation ranges were 6.52–6.83 which are in the range of WHO standards.

### 2. Hardness:
Hardness is mainly caused by calcium and magnesium salts. These salts are dissolved from geologic deposits through which water travels. The length of time water is in contact with hardness producing material helps determine how much hardness there is in raw water. Hardness was originally defined as the capacity of water to precipitate soap caused by Calcium and Magnesium.

### 3. Solids (Total dissolved solids - TDS):
Water has the ability to dissolve a wide range of inorganic and some organic minerals or salts such as potassium, calcium, sodium, bicarbonates, chlorides, magnesium, sulfates etc. These minerals produced un-wanted taste and diluted color in appearance of water. This is the important parameter for the use of water. The water with high TDS value indicates that water is highly mineralized. Desirable limit for TDS is 500 mg/l and maximum limit is 1000 mg/l which prescribed for drinking purpose.

### 4. Chloramines:
Chlorine and chloramine are the major disinfectants used in public water systems. Chloramines are most commonly formed when ammonia is added to chlorine to treat drinking water. Chlorine levels up to 4 milligrams per liter (mg/L or 4 parts per million (ppm)) are considered safe in drinking water.

### 5. Sulfate:
Sulfates are naturally occurring substances that are found in minerals, soil, and rocks. They are present in ambient air, groundwater, plants, and food. The principal commercial use of sulfate is in the chemical industry. Sulfate concentration in seawater is about 2,700 milligrams per liter (mg/L). It ranges from 3 to 30 mg/L in most freshwater supplies, although much higher concentrations (1000 mg/L) are found in some geographic locations.

### 6. Conductivity:
Pure water is not a good conductor of electric current rather’s a good insulator. Increase in ions concentration enhances the electrical conductivity of water. Generally, the amount of dissolved solids in water determines the electrical conductivity. Electrical conductivity (EC) actually measures the ionic process of a solution that enables it to transmit current. According to WHO standards, EC value should not exceeded 400 μS/cm.

### 7. Organic_carbon:
Total Organic Carbon (TOC) in source waters comes from decaying natural organic matter (NOM) as well as synthetic sources. TOC is a measure of the total amount of carbon in organic compounds in pure water. According to US EPA < 2 mg/L as TOC in treated / drinking water, and < 4 mg/Lit in source water which is use for treatment.

### 8. Trihalomethanes:
THMs are chemicals which may be found in water treated with chlorine. The concentration of THMs in drinking water varies according to the level of organic material in the water, the amount of chlorine required to treat the water, and the temperature of the water that is being treated. THM levels up to 80 ppm is considered safe in drinking water.

### 9. Turbidity:
The turbidity of water depends on the quantity of solid matter present in the suspended state. It is a measure of light emitting properties of water and the test is used to indicate the quality of waste discharge with respect to colloidal matter. The mean turbidity value obtained for Wondo Genet Campus (0.98 NTU) is lower than 0the WHO recommended value of 5.00 NTU.

### 10. Potability:
Indicates if water is safe for human consumption where 1 means Potable and 0 means Not potable.



In [ ]:
water.sample(5)

In [ ]:
print(f'shape of  dataset {water.shape[0],water.shape[-1]}')

## sort view  of columns of dataset

In [ ]:
for i in range(len(water.columns)):
  print(water.columns[i])

In [ ]:
water.info()

In [ ]:
water.isnull().sum()

In [ ]:
water.describe().T.style.bar(subset=['mean'],color='Reds').background_gradient(subset='std',cmap='ocean').background_gradient(subset=['50%'],cmap='plasma').background_gradient(subset=['max'],cmap=None)

# outliers of the dataset

In [ ]:
def outlier_check(var):
    sns.displot(data = water,x=var )
    print(" ")
    print("Variable : ",var)
    print("Minimum value :",water[var].min())
    print("1st percentile :",water[var].quantile(0.01))
    print("99th percentitle :",water[var].quantile(0.99))
    print("Maximum value :",water[var].max())

In [ ]:
for i in range(len(water.columns)):
  outlier_check(water.columns[i])

In [ ]:
sns.pairplot(water)

In [ ]:
sns.countplot(water['Potability'])
print(pd.concat([water['Potability'].value_counts(),water['Potability'].value_counts(normalize=True).mul(100).round(2)],axis=1,keys=('Count','Percentage')))

In [ ]:
plt.figure(figsize=(12, 24))
heatmap = sns.heatmap(water.corr()[['Potability']].sort_values(by='Potability', ascending=True), 
                      vmin=-1, 
                      vmax=1, 
                      annot=True, 
                      cmap=sns.diverging_palette(5, 5, as_cmap=True))
heatmap.set_title('Features Correlating with Target', fontdict={'fontsize':18}, pad=16)

In [ ]:
low_corr_vars =['Trihalomethanes','Turbidity','ph','Conductivity','Sulfate']
water.drop(low_corr_vars,axis=1,inplace=True)

In [ ]:
water.head()

In [ ]:
water.isnull().sum()

In [ ]:
corr_water = water.drop(['Potability'],axis=1)
corrmat = corr_water.corr()

In [ ]:
f , ax = plt.subplots(figsize=(15,15))
sns.heatmap(corrmat.round(2),vmin=-1,vmax=1,square=True,annot=True,cmap=sns.diverging_palette(5,5,as_cmap=True))

In [ ]:
X =  water.drop('Potability',axis=1)
y =  water['Potability']

In [ ]:

X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=1)

In [ ]:
print('Train data records : %d\nTest data records :%d'%(X_train.shape[0] ,X_test.shape[0]))

In [ ]:
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train,y_train)

In [ ]:
train_predictions = logistic_model.predict(X_train)
test_predictions = logistic_model.predict(X_test)

In [ ]:
model_performance = pd.DataFrame([['LogisticRegression',round(accuracy_score(y_train,train_predictions)*100,2),round(accuracy_score(y_test,test_predictions)*100,2)]],columns=['Model','Train_Accuracy','Test_Accuracy'])

In [ ]:
print(model_performance)

In [ ]:
def dt_score(max_leaf_nodes, X_train, y_train,X_test,y_test):
    model = DecisionTreeClassifier(max_leaf_nodes= max_leaf_nodes,random_state=1)
    model.fit(X_train,y_train)
    train_predict= model.predict(X_train)
    test_predict = model.predict(X_test)
    ac_score_train = round(accuracy_score(y_train,train_predict)*100,2)
    ac_score_test = round(accuracy_score(y_test,test_predict)*100,2)
    return(ac_score_train,ac_score_test)

In [ ]:
dt_score_results = pd.DataFrame(columns=['Max_leaf_nodes','Train_Accuracy','Test_Accuracy'])
for max_leaf_nodes in [2,3,4,5,10,20,30,40,50]:
    ac_score_train,ac_score_test = dt_score(max_leaf_nodes,X_train,y_train,X_test,y_test)
    print("Max leaf nodes: %d  \t Train Accuracy:  %.2f \t Test Accuracy %.2f" % (max_leaf_nodes, ac_score_train, ac_score_test))

In [ ]:
decision_tree_model = DecisionTreeClassifier(max_leaf_nodes=50,random_state=1)
decision_tree_model.fit(X_train,y_train)
train_prediction = decision_tree_model.predict(X_train)
test_predictions = decision_tree_model.predict(X_test)
model_performance = model_performance.append(pd.DataFrame([['DecisionTreeClassifier',round(accuracy_score(y_train,train_predictions)*100,2),round(accuracy_score(y_test,test_predictions)*100,2)]],columns=['Model','Train_Accuracy','Test_Accuracy']),ignore_index=True)
print(model_performance)

In [ ]:
def rf_score(max_leaf_nodes, X_train, y_train,X_test,y_test):
    model = RandomForestClassifier(max_leaf_nodes= max_leaf_nodes,random_state=1)
    model.fit(X_train,y_train)
    train_predict= model.predict(X_train)
    test_predict = model.predict(X_test)
    ac_score_train = round(accuracy_score(y_train,train_predict)*100,2)
    ac_score_test = round(accuracy_score(y_test,test_predict)*100,2)
    return(ac_score_train,ac_score_test)

In [ ]:
rf_score_results = pd.DataFrame(columns=['Max_leaf_nodes','Train_Accuracy','Test_Accuracy'])
for max_leaf_nodes in [2,3,4,5,10,20,30,40,50]:
    ac_score_train,ac_score_test = rf_score(max_leaf_nodes,X_train,y_train,X_test,y_test)
    print("Max leaf nodes: %d  \t Train Accuracy:  %.2f \t Test Accuracy %.2f" % (max_leaf_nodes, ac_score_train, ac_score_test))

In [ ]:
random_forest_model = RandomForestClassifier(max_leaf_nodes=50,random_state=1)
random_forest_model.fit(X_train,y_train)
train_prediction = decision_tree_model.predict(X_train)
test_predictions = decision_tree_model.predict(X_test)
model_performance = model_performance.append(pd.DataFrame([['RandomForestClassifier',round(accuracy_score(y_train,train_predictions)*100,2),round(accuracy_score(y_test,test_predictions)*100,2)]],columns=['Model','Train_Accuracy','Test_Accuracy']),ignore_index=True)
print(model_performance)

In [ ]:
def gb_score(max_leaf_nodes, X_train, y_train,X_test,y_test):
    model = GradientBoostingClassifier(max_leaf_nodes= max_leaf_nodes,random_state=1)
    model.fit(X_train,y_train)
    train_predict= model.predict(X_train)
    test_predict = model.predict(X_test)
    ac_score_train = round(accuracy_score(y_train,train_predict)*100,2)
    ac_score_test = round(accuracy_score(y_test,test_predict)*100,2)
    return(ac_score_train,ac_score_test)

In [ ]:
gb_score_results = pd.DataFrame(columns=['Max_leaf_nodes','Train_Accuracy','Test_Accuracy'])
for max_leaf_nodes in [2,3,4,5,10,20,30,40,50]:
    ac_score_train,ac_score_test = gb_score(max_leaf_nodes,X_train,y_train,X_test,y_test)
    print("Max leaf nodes: %d  \t Train Accuracy:  %.2f \t Test Accuracy %.2f" % (max_leaf_nodes, ac_score_train, ac_score_test))

In [ ]:
gb_model = GradientBoostingClassifier(max_leaf_nodes= 10,random_state=1)
gb_model.fit(X_train,y_train)
train_prediction = decision_tree_model.predict(X_train)
test_predictions = decision_tree_model.predict(X_test)
model_performance = model_performance.append(pd.DataFrame([['GradientBoostingClassifier',round(accuracy_score(y_train,train_predictions)*100,2),round(accuracy_score(y_test,test_predictions)*100,2)]],columns=['Model','Train_Accuracy','Test_Accuracy']),ignore_index=True)
print(model_performance)

In [ ]:
full_predictions = gb_model.predict(X_test)
print( "Accuracy Score: %.2f " % (round(accuracy_score(y_test, full_predictions)*100,2)))

# Plotting confusion matrix
plot_confusion_matrix(random_forest_model,
                      X_test, 
                      y_test,
                      cmap=plt.cm.Blues,
                      normalize= 'true')